In [7]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer, recall_score, f1_score, accuracy_score
import numpy as np
from imblearn.metrics import geometric_mean_score
import pandas as pd
from collections import Counter
from tqdm import tqdm
import seaborn as sns

In [13]:
predictions = pd.read_csv('unlabeled_predictions_oversampling.csv')
vt_labels = pd.read_csv("vt_labels.csv")
#labels_1 = pd.read_csv("vt_labels_1.csv").Labels.values.tolist()

In [4]:
def specificity(y_true, y_pred): 
    tn = confusion_matrix(y_true, y_pred)[0, 0]
    fp = confusion_matrix(y_true, y_pred)[0, 1]
    neg = tn + fp
    return tn/neg


In [14]:
len(vt_labels)

179117

In [22]:
#predictions = pd.concat([vt_labels.Domains, predictions], axis=1)
#predictions.to_csv('unlabeled_predictions_2_doms.csv', index=False, header=True)

In [15]:
predictions = predictions[:179117]

In [16]:
predictions.drop(vt_labels[vt_labels.Label == 'not_found'].index, inplace=True)
vt_labels.drop(vt_labels[vt_labels.Label == 'not_found'].index, inplace=True)

In [17]:
predictions.reset_index(drop=True, inplace=True)
vt_labels.reset_index(drop=True, inplace=True)

In [20]:
vt_labels.tail()

,Domain,Label
88674,figlidelcorona.com,1
88675,figlidelcorona.eu,1
88676,figlidelcorona.it,1
88677,figshare.com,1
88678,figueirao.ms.gov.br,0


In [21]:
predictions.tail()

,dtc_bsmt,rfc_bsmt,xgb_bsmt,gbc_bsmt,mlp_bsmt,svc_bsmt,dtc_bsvm,rfc_bsvm,xgb_bsvm,gbc_bsvm,mlp_bsvm,svc_bsvm,dtc_bADA,rfc_bADA,xgb_bADA,gbc_bADA,mlp_bADA,svc_bADA
88674,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
88675,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
88676,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
88677,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0
88678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
labels = vt_labels.Label.values.tolist()
labels = [int(i) for i in labels]

In [24]:
#ht_sd = predictions['Labeled with subdomains'].values.tolist()
#ht_nosd = predictions['Labeled w/o subdomains'].values.tolist()
dtc_bsmt = predictions['dtc_bsmt'].values.tolist()
rfc_bsmt = predictions['rfc_bsmt'].values.tolist()
xgb_bsmt = predictions['xgb_bsmt'].values.tolist()
gbc_bsmt = predictions['gbc_bsmt'].values.tolist()
mlp_bsmt = predictions['mlp_bsmt'].values.tolist()
svc_bsmt = predictions['svc_bsmt'].values.tolist()

dtc_bsvm = predictions['dtc_bsvm'].values.tolist()
rfc_bsvm = predictions['rfc_bsvm'].values.tolist()
xgb_bsvm = predictions['xgb_bsvm'].values.tolist()
gbc_bsvm = predictions['gbc_bsvm'].values.tolist()
mlp_bsvm = predictions['mlp_bsvm'].values.tolist()
svc_bsvm = predictions['svc_bsvm'].values.tolist()

dtc_bADA = predictions['dtc_bADA'].values.tolist()
rfc_bADA = predictions['rfc_bADA'].values.tolist()
xgb_bADA = predictions['xgb_bADA'].values.tolist()
gbc_bADA = predictions['gbc_bADA'].values.tolist()
mlp_bADA = predictions['mlp_bADA'].values.tolist()
svc_bADA = predictions['svc_bADA'].values.tolist()

In [ ]:
predictions.drop('Domain', inplace=True, axis=1)

In [26]:
print(Counter(labels))

Counter({1: 51954, 0: 36725})


In [27]:
len(labels)

88679

In [36]:
kk_0 = []
for i in predictions.columns:
    kk_0.append(specificity(labels, predictions[i].values.tolist()))
    

kk_0 = pd.Series(kk_0, name='Recall score against VT labels')

i = 0
for j in predictions.columns:
    kk_0.rename(index={i: j}, inplace=True)
    i += 1

In [37]:
pd.concat([kk_0], axis=1)

,Recall score against VT labels
dtc_bsmt,0.556814
rfc_bsmt,0.574268
xgb_bsmt,0.579170
gbc_bsmt,0.647597
mlp_bsmt,0.582573
svc_bsmt,0.605718
dtc_bsvm,0.540231
rfc_bsvm,0.582600
xgb_bsvm,0.581021
gbc_bsvm,0.612417


In [ ]:
kk_1 = []
for i in predictions.columns:
    kk_1.append(cohen_kappa_score(ht_sd, predictions[i].values.tolist()[:30553]))

kk_1 = pd.Series(kk_1, name='HT_sd')

In [18]:
kk_2 = []
for i in predictions.columns:
    kk_2.append(cohen_kappa_score(ht_nosd, predictions[i].values.tolist()[:30553]))

kk_2 = pd.Series(kk_2, name='HT_nosd')

In [19]:
kk_3 = []
for i in predictions.columns:
    kk_3.append(cohen_kappa_score(dtc, predictions[i].values.tolist()[:30553]))
    
kk_3 = pd.Series(kk_3, name='DTC_nosd')

In [20]:
kk_4 = []
for i in predictions.columns:
    kk_4.append(cohen_kappa_score(rfc, predictions[i].values.tolist()[:30553]))
    
kk_4 = pd.Series(kk_4, name='RFC_nosd')

In [21]:
kk_5 = []
for i in predictions.columns:
    kk_5.append(cohen_kappa_score(xgb, predictions[i].values.tolist()[:30553]))
kk_5 = pd.Series(kk_5, name='XGB_nosd')

In [22]:
kk_6 = []
for i in predictions.columns:
    kk_6.append(cohen_kappa_score(gbc, predictions[i].values.tolist()[:30553]))
kk_6 = pd.Series(kk_6, name='GBC_nosd')

In [23]:
kk_7 = []
for i in predictions.columns:
    kk_7.append(cohen_kappa_score(mlp, predictions[i].values.tolist()[:30553]))
kk_7 = pd.Series(kk_7, name='MLP_nosd')

In [24]:
kk_8 = []
for i in predictions.columns:
    kk_8.append(cohen_kappa_score(svc, predictions[i].values.tolist()[:30553]))
kk_8 = pd.Series(kk_8, name='SVC_nosd')

In [25]:
kk_9 = []
for i in predictions.columns:
    kk_9.append(cohen_kappa_score(dtc_sd, predictions[i].values.tolist()[:30553]))
kk_9 = pd.Series(kk_9, name='DTC_sd')

In [26]:
kk_10 = []
for i in predictions.columns:
    kk_10.append(cohen_kappa_score(rfc_sd, predictions[i].values.tolist()[:30553]))
kk_10 = pd.Series(kk_10, name='RFC_sd')

In [27]:
kk_11 = []
for i in predictions.columns:
    kk_11.append(cohen_kappa_score(xgb_sd, predictions[i].values.tolist()[:30553]))
kk_11 = pd.Series(kk_11, name='XGB_sd')

In [28]:
kk_12 = []
for i in predictions.columns:
    kk_12.append(cohen_kappa_score(gbc_sd, predictions[i].values.tolist()[:30553]))
kk_12 = pd.Series(kk_12, name='GBC_sd')

In [29]:
kk_13 = []
for i in predictions.columns:
    kk_13.append(cohen_kappa_score(mlp_sd, predictions[i].values.tolist()[:30553]))
kk_13 = pd.Series(kk_13, name='MLP_sd')

In [30]:
kk_14 = []
for i in predictions.columns:
    kk_14.append(cohen_kappa_score(svc_sd, predictions[i].values.tolist()[:30553]))
kk_14 = pd.Series(kk_14, name='SVC_sd')

In [31]:
scores = pd.concat([kk_1, kk_2, kk_3, kk_4, kk_5, kk_6, kk_7, kk_8, kk_9, kk_10, kk_11, kk_12, kk_13, kk_14], axis=1)

In [32]:
i = 0
for j in scores.columns:
    scores.rename(index={i: j}, inplace=True)
    i += 1

In [33]:
# at 30553
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.733355,0.777624,0.845932,0.851883,0.836810,0.906817,0.859882,0.951362,0.897768,0.891480,0.909447,0.912916,0.953949
HT_nosd,0.733355,1.000000,0.695464,0.761841,0.753023,0.804358,0.774141,0.784409,0.770997,0.738959,0.742047,0.744479,0.759810,0.773056
DTC_nosd,0.777624,0.695464,1.000000,0.849142,0.848101,0.807258,0.820421,0.787973,0.819595,0.845113,0.833160,0.828802,0.810710,0.819461
RFC_nosd,0.845932,0.761841,0.849142,1.000000,0.923542,0.891981,0.900799,0.880820,0.891123,0.928202,0.911579,0.894432,0.889071,0.890466
XGB_nosd,0.851883,0.753023,0.848101,0.923542,1.000000,0.889695,0.903178,0.862273,0.897205,0.919874,0.934669,0.903853,0.895275,0.898013
GBC_nosd,0.836810,0.804358,0.807258,0.891981,0.889695,1.000000,0.891212,0.869533,0.880197,0.870561,0.879799,0.886916,0.877941,0.880965
MLP_nosd,0.906817,0.774141,0.820421,0.900799,0.903178,0.891212,1.000000,0.912124,0.951643,0.924997,0.924674,0.935677,0.969362,0.952391
SVC_nosd,0.859882,0.784409,0.787973,0.880820,0.862273,0.869533,0.912124,1.000000,0.901736,0.873408,0.875313,0.887197,0.906824,0.904329
DTC_sd,0.951362,0.770997,0.819595,0.891123,0.897205,0.880197,0.951643,0.901736,1.000000,0.944686,0.937542,0.955857,0.959787,0.996948
RFC_sd,0.897768,0.738959,0.845113,0.928202,0.919874,0.870561,0.924997,0.873408,0.944686,1.000000,0.954896,0.944292,0.935728,0.944476


In [23]:
# at 10000
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.472702,0.536007,0.635022,0.641643,0.622850,0.759843,0.602941,0.833846,0.720133,0.717321,0.742215,0.777479,0.833846
HT_nosd,0.472702,1.000000,0.488778,0.538546,0.489606,0.579053,0.529465,0.631339,0.543579,0.457043,0.468487,0.474360,0.515412,0.543579
DTC_nosd,0.536007,0.488778,1.000000,0.676990,0.694215,0.661483,0.612966,0.547179,0.646421,0.692239,0.694215,0.676805,0.607393,0.646421
RFC_nosd,0.635022,0.538546,0.676990,1.000000,0.841810,0.874899,0.761807,0.725382,0.763219,0.828391,0.808507,0.783177,0.752192,0.763219
XGB_nosd,0.641643,0.489606,0.694215,0.841810,1.000000,0.817004,0.787290,0.647358,0.782216,0.838146,0.859107,0.800113,0.771227,0.782216
GBC_nosd,0.622850,0.579053,0.661483,0.874899,0.817004,1.000000,0.744904,0.728654,0.747071,0.749378,0.756915,0.754506,0.730170,0.747071
MLP_nosd,0.759843,0.529465,0.612966,0.761807,0.787290,0.744904,1.000000,0.738204,0.919553,0.841608,0.838340,0.883735,0.960071,0.919553
SVC_nosd,0.602941,0.631339,0.547179,0.725382,0.647358,0.728654,0.738204,1.000000,0.711062,0.636694,0.633619,0.660827,0.725705,0.711062
DTC_sd,0.833846,0.543579,0.646421,0.763219,0.782216,0.747071,0.919553,0.711062,1.000000,0.876638,0.873632,0.900143,0.941758,1.000000
RFC_sd,0.720133,0.457043,0.692239,0.828391,0.838146,0.749378,0.841608,0.636694,0.876638,1.000000,0.929761,0.910755,0.860261,0.876638


In [70]:
# at 350000 (original)
scores

,HT_sd,HT_nosd,DTC_nosd,RFC_nosd,XGB_nosd,GBC_nosd,MLP_nosd,SVC_nosd,DTC_sd,RFC_sd,XGB_sd,GBC_sd,MLP_sd,SVC_sd
HT_sd,1.000000,0.380922,0.223846,0.219961,0.230166,0.205385,0.251964,0.234591,0.282199,0.249172,0.247880,0.247925,0.250202,0.283565
HT_nosd,0.380922,1.000000,0.168401,0.182236,0.189823,0.174776,0.196362,0.194510,0.206367,0.196916,0.197348,0.199779,0.202501,0.206662
DTC_nosd,0.223846,0.168401,1.000000,0.840180,0.833158,0.772407,0.801836,0.786061,0.796798,0.817793,0.808683,0.803868,0.786245,0.796271
RFC_nosd,0.219961,0.182236,0.840180,1.000000,0.925738,0.858297,0.897131,0.890813,0.879436,0.912516,0.898605,0.878642,0.875025,0.879449
XGB_nosd,0.230166,0.189823,0.833158,0.925738,1.000000,0.843979,0.915568,0.892551,0.899669,0.916257,0.930267,0.896692,0.892036,0.899631
GBC_nosd,0.205385,0.174776,0.772407,0.858297,0.843979,1.000000,0.848544,0.852477,0.825274,0.817670,0.821906,0.826516,0.823985,0.825561
MLP_nosd,0.251964,0.196362,0.801836,0.897131,0.915568,0.848544,1.000000,0.945947,0.957674,0.925052,0.928809,0.931833,0.956917,0.958294
SVC_nosd,0.234591,0.194510,0.786061,0.890813,0.892551,0.852477,0.945947,1.000000,0.934562,0.907943,0.906774,0.917559,0.937460,0.935943
DTC_sd,0.282199,0.206367,0.796798,0.879436,0.899669,0.825274,0.957674,0.934562,1.000000,0.952813,0.950820,0.956459,0.967228,0.998285
RFC_sd,0.249172,0.196916,0.817793,0.912516,0.916257,0.817670,0.925052,0.907943,0.952813,1.000000,0.962518,0.948853,0.942135,0.952739
